#Resources
1. Open CV https://www.geeksforgeeks.org/python/python-program-extract-frames-using-opencv/
2. 

In [3]:
#installs
pip install opencv-python

SyntaxError: invalid syntax (1972351035.py, line 2)

In [ ]:
#imports
from datetime import timedelta
import cv2
import numpy as np
import os

OpenCV Version 1 (Geeks to Geeks)

In [2]:
vid = cv2.VideoCapture("C:\\Users\\Admin\\PycharmProjects\\project_1\\openCV.mp4")

count, success = 0, True
while success:
    success, image = vid.read() # Read frame
    if success: 
        cv2.imwrite(f"frame{count}.jpg", image) # Save frame
        count += 1

vid.release()

ModuleNotFoundError: No module named 'cv2'

Open CV Version 2

In [4]:
#Since not all videos have the same length and FPS, 
#we will define a parameter to adjust how many frames we want to extract and save per second:
#We will use this parameter in both methods. For instance, if it's set to 10 for now, 
#it will save only 10 frames per second of the video, even though the video FPS is, say, 24.
#If the video is 30 seconds long, then 300 frames will be saved in total.
#You can also set this parameter to, say, 0.5, which will save one frame per 2 seconds, and so on.

# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 10


Next, let's define two helper functions:

In [5]:
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

The format_timedelta() function accepts a timedelta object, returns a nice string representation with milliseconds, and omits the microseconds.

The get_saving_frames_durations() function accepts the VideoCapture object from OpenCV, and the saving parameter we discussed earlier and returns a list of duration spots on where we should save the frames.

Now that we have these helper functions, let's define the main function and explain it:

In [6]:
def main(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # read the video file    
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), frame) 
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1

In [7]:
The above function looks complicated, but it's not; here's what we're doing:

    First, we make the filename variable, which is the folder name we're going to create and save our frames in; we append "-opencv" just to distinguish the methods, but you can delete them.
    Then, we create the folder using the os.mkdir() function if not already created.
    After that, we read the video file using cv2.VideoCapture, and retrieves the FPS using the cap.get() method and pass the code for FPS, which is cv2.CAP_PROP_FPS.
    We set the saving frames per second to the minimum of the actual video FPS and our parameter. So, we ensure we cannot bypass a higher saving fps than the actual video fps.
    After we get the saving durations, we enter the loop of reading the frames, and we only save when we're sure that the duration is in our saving_frames_durations list. We save the frame using cv2.imwrite(), and set the frame name to the actual duration.

Defining the main code:

SyntaxError: unterminated string literal (detected at line 1) (2379968892.py, line 1)

Define Main

In [8]:
if __name__ == "__main__":
    import sys
    video_file = sys.argv[1]
    main(video_file)

NameError: name 'os' is not defined

In [ ]:
Pass video via command line arguments

In [ ]:
$ python extract_frames_opencv.py zoo.mp4